In [ ]:
import numpy as np
import pandas as pd
import os
from joblib import Parallel,delayed

In [ ]:
# testing module

import matplotlib.pyplot as plt

code="601211.SH"
date=20191023
pre_date=20191022

close_seq=np.array(data.loc[(code,date)]["close"])
pre_close_seq=np.array(data.loc[(code,pre_date)]["close"][-60:])
pre_close=data.loc[(code,date)]["pre_close"][0]
today_open=data.loc[(code,date)]["open"][0]

plt.plot(pre_close_seq)
plt.grid()
plt.show()

plt.plot(close_seq)
plt.grid()
plt.show()

print(cross_count(pre_close_seq,close_seq,pre_close))


In [45]:
#!/home/ywang/env/dev/bin
#-*-coding:utf-8-*-
# written by wangyonghao

# the api: get_daily_cross_data(date)

# input:  date: current date ( "20190102" for example)

# output to file: the csv file of the cross_grid_count for each stock

import numpy as np
import pandas as pd
import os
from joblib import Parallel, delayed
from fb_base_get_market_data import *
from data_tools.api import *
from utilscht.Data import *
import pymysql
import datetime
import logging
import click
from reader.reader import BarReader

def get_cross_grid_info(price_now, ref_price,grids):
    if price_now > ref_price:
        cross_grids = [i for i in grids if i > ref_price and i <= price_now]
        if len(cross_grids) > 0:
            grids_num = len(cross_grids)
            direction = 1
            last_grid = cross_grids[-1]

    else:
        cross_grids = [i for i in grids if i > price_now and i <= ref_price]
        if len(cross_grids) > 0:
            grids_num = len(cross_grids)
            direction = -1
            last_grid = cross_grids[0]

    if len(cross_grids) > 0:
        return (grids_num, direction, last_grid)
    else:
        return -1


# 输入前一天最后 60min 的close序列、今天的全部close序列、前一天的收盘价、今天的开盘价
# ，返回今天穿越网格线的次数（一段时间内重复穿过同一条线只计算一次）
def get_cross_count(pre_close_seq, close_seq):
    total_seq=np.concatenate([pre_close_seq,close_seq])
    if np.isnan(total_seq).sum()>10:
        return np.nan
    
    if (np.nanmin(total_seq)+np.nanmax(total_seq))/2>=10:
        grid_unit=(np.nanmin(total_seq)+np.nanmax(total_seq))/2*0.01
    else:
        grid_unit = 0.1
    grid_num = int((np.nanmax(total_seq)-np.nanmin(total_seq))/grid_unit+3)
    grids = np.linspace(np.nanmin(total_seq)-grid_unit,np.nanmax(total_seq)+grid_unit,grid_num)
    
    count = 0
    ref_price = 0
    pre_cross_dire = 1

    # get the last corss_line for yesterday
    for i in reversed(range(len(pre_close_seq))):
        price_pre = pre_close_seq[i - 1]
        price_now = pre_close_seq[i]
        cross_grid_info = get_cross_grid_info(price_now, price_pre,grids)

        if cross_grid_info != -1:
            ref_price = cross_grid_info[2]
            pre_cross_dire = cross_grid_info[1]
            break

    # count the cross times for today
    close_seq = np.concatenate([np.array([pre_close_seq[-1]]),close_seq])
    for i in range(len(close_seq) - 1):
        price_pre = close_seq[i]
        price_now = close_seq[i + 1]

        cross_grid_info = get_cross_grid_info(price_now, price_pre,grids)

        if cross_grid_info == -1:
            continue

        # judge the status of "now" and "pre"
        grids_num = cross_grid_info[0]
        cross_dire = cross_grid_info[1]
        last_grid = cross_grid_info[2]

        if last_grid != ref_price:
            if cross_dire * pre_cross_dire == 1:
                count = count + grids_num
            else:
                count = count + grids_num - 1

        ref_price = last_grid
        pre_cross_dire = cross_dire

    return count


# current function to get 1min_freq data
def get_stk_data(sid, date):
    
    try:
        DTYPE_STK_BAR_1MIN = [
            ('wcode', 'S16'),
            ('date', 'i4'),
            ('time', 'i4'),
            ('open', 'i4'),
            ('high', 'i4'),
            ('low', 'i4'),
            ('close', 'i4'),
            ('volume', 'i8'),
            ('value', 'i8'),
            ('num_trades', 'i4'),
            ('unused', 'i4'),
        ]



        bar=np.memmap(r"/dat/raw/wind/stk_bar_1min/{}/{}.dat".format(date,sid),dtype=DTYPE_STK_BAR_1MIN,mode="r")
        df = pd.DataFrame(bar)[["wcode","date","time","close"]]
        df["sid"] = df["wcode"].apply(lambda x:str(x)[2:-1])
        df["DataDate"] = df["date"].apply(str)
        df["close"] = df["close"]/10000
        df["pre_close"] = get_stk_bar(sid, freq="1d", start=date, end=date, fields=["pre_close"]).values[0][0]
        df = df.drop(["wcode","date","time"],axis=1)
        df["ticktime"] = df.index
        return df[["sid","DataDate","ticktime","close","pre_close"]]
    except Exception as e:
        try:
            stock_data = get_stk_bar(sid, freq="1m", start=date, end=date, fields=["open", "high", "low", "close"])
            stock_data = stock_data.reset_index().rename(columns={"index": "datetime"})
            stock_data["ticktime"] = stock_data["datetime"].index
            stock_data["DataDate"] = stock_data["datetime"].apply(lambda x:str(x.date()))
            del stock_data["datetime"]
            stock_data["pre_close"] = get_stk_bar(sid, freq="1d", start=date, end=date, fields=["pre_close"]).values[0][0]
            stock_data["sid"] = sid
            return stock_data
        except:
            logging.warning("Data Lost for {} on {}".format(sid,date))


def get_daily_data(date):
    df=get_db(start_date = date, end_date = date,cols=["trade_status"])
    stk_ls=list(df["sid"][df["trade_status"]==1])

    results = Parallel(n_jobs=16, verbose=5, backend="loky", batch_size='auto') \
        (delayed(get_stk_data)(sid, date) for sid in stk_ls)
    data = pd.concat(results)

    return data.sort_values(["sid", "ticktime"])

def get_today_stk_data(sid,date):
    try:
        with BarReader('/home/ywang/proj_cross_grid/reader/bar.yaml') as client:
            bars = client.stock_bars(sid[0:6])
            df = pd.DataFrame(bars)
            df["sid"] = df["symbol"].apply(lambda x:str(x)[2:-1])
            df["DataDate"] = df["date"].apply(str)
            df["ticktime"] = df.index
            df = df.drop(["symbol","date","time"],axis=1)
            df["pre_close"] = df["preclose"].iloc[0]
            df= df[df["sid"] != ""]
            return df[["sid","DataDate","ticktime","close","pre_close"]]
    except:
        logging.warning("Data Lost for {} on {}".format(sid,date))


def get_today_data(date):
    pre_date = get_previous_trade_date(date)
    df=get_db(start_date = pre_date, end_date = pre_date,cols=["trade_status"])
    stk_ls=list(df["sid"][df["trade_status"] == 1])

    results = Parallel(n_jobs=16, verbose=5, backend="loky", batch_size='auto') \
        (delayed(get_today_stk_data)(sid,date) for sid in stk_ls)
    data = pd.concat(results)

    return data.sort_values(["sid", "ticktime"])


def get_count_parallel(df):
    if sum(pd.isnull(df["yesterday_close"]))>120:
        return pd.DataFrame()
    sid = df["sid"].drop_duplicates()
    temp_df = pd.DataFrame(index=sid, columns=["cross_count"])
    
    pre_close = df["pre_close"].iloc[0]
    if pre_close!=df["yesterday_close"].iloc[-1]:
        df["yesterday_close"] = df["yesterday_close"]* pre_close/df["yesterday_close"].iloc[-1]

    close_seq = np.concatenate([np.array(df["yesterday_close"].iloc[-40:]),\
                                np.array(df["close"].iloc[:200])])
    close_seq = close_seq[~np.isnan(close_seq)]
    pre_close_seq = np.array(df["yesterday_close"].iloc[-240:-40])
    
    count = get_cross_count(pre_close_seq, close_seq)

    temp_df.iloc[0, 0] = count
    return temp_df


def apply_parallel(df_grouped, func, n_jobs=16, backend='loky', as_index=False, **kwargs):
    """
    This is for parallel between grouped generated by pd.DataFrame.groupby
    :param df_grouped:
    :param func:
    :param n_jobs:
    :param backend:
    :param kwargs:
    :return:
    """

    names = []
    groups = []
    for name, group in df_grouped:
        names.append(name)
        groups.append(group)

    results = Parallel(n_jobs=n_jobs, verbose=5, backend=backend, batch_size='auto') \
        (delayed(func)(group, **kwargs) for group in groups)

    return pd.concat(results, keys=names if as_index else None)

@click.command()
@click.argument("date",nargs=1)
def get_daily_cross_data(date):
    pre_date = get_previous_trade_date(date)

    data = get_today_data(date)
    data["sid"] = data["sid"].apply(lambda x:str(x)+".SH" if str(x)[0]=='6' else str(x)+'.SZ')
    data["DataDate"] = data["DataDate"].apply(str)
    yesterday_data = get_daily_data(pre_date)
    yesterday_data.rename(columns={"close": "yesterday_close"}, inplace=True)
    data = pd.merge(data, yesterday_data[["sid", "ticktime", "yesterday_close"]], how="left", on=["sid", "ticktime"])

    groups = data.groupby("sid")
    cross_count_df = apply_parallel(groups, get_count_parallel)
    cross_count_df.to_csv(r"/home/ywang/proj_cross_grid/result/cross_count_b240/crosscount_{}.csv".format(date), index_label="sid")    

def get_daily_cross_data_hist(date):
    pre_date = get_previous_trade_date(date)

    data = get_daily_data(date)
    yesterday_data = get_daily_data(pre_date)
    yesterday_data.rename(columns={"close": "yesterday_close"}, inplace=True)
    data = pd.merge(data, yesterday_data[["sid", "ticktime", "yesterday_close"]], how="left", on=["sid", "ticktime"])
    
    print(data.head())
    
    groups = data.groupby("sid")
    cross_count_df = apply_parallel(groups, get_count_parallel)
    cross_count_df.to_csv(r"/home/ywang/proj_cross_grid/result/cross_count_b240/crosscount_{}.csv".format(date), index_label="sid")    


In [49]:
get_daily_cross_data_hist("20200521")

[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  40 tasks      | elapsed:    1.7s
[Parallel(n_jobs=16)]: Done 216 tasks      | elapsed:    2.5s
[Parallel(n_jobs=16)]: Done 720 tasks      | elapsed:    4.4s
[Parallel(n_jobs=16)]: Done 1368 tasks      | elapsed:    6.9s
[Parallel(n_jobs=16)]: Done 2160 tasks      | elapsed:    9.9s
[Parallel(n_jobs=16)]: Done 3096 tasks      | elapsed:   13.6s
[Parallel(n_jobs=16)]: Done 3783 out of 3814 | elapsed:   16.0s remaining:    0.1s
[Parallel(n_jobs=16)]: Done 3814 out of 3814 | elapsed:   16.0s finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  40 tasks      | elapsed:    1.6s
[Parallel(n_jobs=16)]: Done 184 tasks      | elapsed:    2.3s
[Parallel(n_jobs=16)]: Done 688 tasks      | elapsed:    4.3s
[Parallel(n_jobs=16)]: Done 1336 tasks      | elapsed:    6.8s
[Parallel(n_jobs=16)]: Done 2128 tasks      | elapsed:    9.8s
[Pa

         sid  DataDate  ticktime  close  pre_close  yesterday_close
0  000001.SZ  20200521         0  13.54      13.51            13.32
1  000001.SZ  20200521         1  13.52      13.51            13.30
2  000001.SZ  20200521         2  13.54      13.51            13.27
3  000001.SZ  20200521         3  13.51      13.51            13.30
4  000001.SZ  20200521         4  13.50      13.51            13.29


[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  40 tasks      | elapsed:    1.7s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:    2.3s
[Parallel(n_jobs=16)]: Done 672 tasks      | elapsed:    3.5s
[Parallel(n_jobs=16)]: Done 1320 tasks      | elapsed:    5.0s
[Parallel(n_jobs=16)]: Done 2112 tasks      | elapsed:    7.0s
[Parallel(n_jobs=16)]: Done 3048 tasks      | elapsed:    9.6s
[Parallel(n_jobs=16)]: Done 3707 out of 3707 | elapsed:   11.4s finished


In [37]:
def apply_parallel(df_grouped, func, n_jobs=16, backend='loky', as_index=True, **kwargs):
    """
    This is for parallel between grouped generated by pd.DataFrame.groupby
    :param df_grouped:
    :param func:
    :param n_jobs:
    :param backend:
    :param kwargs:
    :return:
    """

    names = []
    groups = []
    for name, group in df_grouped:
        names.append(name)
        groups.append(group)

    results = Parallel(n_jobs=n_jobs, verbose=5, backend=backend, batch_size='auto') \
        (delayed(func)(group, **kwargs) for group in groups)

    return pd.concat(results, keys=names if as_index else None)



def get_period_stk_data(sid,start_date,end_date):
    try:
        stock_data = get_stk_bar(sid, freq="1m", start=start_date, end=end_date, fields=["open", "high", "low", "close"])
        stock_data = stock_data.reset_index().rename(columns={"index": "datetime"})
        stock_data["DataDate"]=stock_data["datetime"].apply(lambda x:str(x)[0:10].replace('-',''))
        df_pre_close = get_stk_bar(sid, freq="1d", start=start_date, end=end_date, fields=["pre_close"])
        df_pre_close = df_pre_close.reset_index().rename(columns={"date": "datetime"})
        df_pre_close["DataDate"]=df_pre_close["datetime"].apply(lambda x:str(x)[0:10].replace('-',''))
        df_pre_close = df_pre_close.drop(["datetime"],axis=1)
        stock_data = stock_data.merge(df_pre_close,on="DataDate")
        stock_data["sid"] = sid
    except Exception as e:
        print(e)
        return pd.DataFrame()
    return stock_data

def get_period_data(start_date, end_date):
    df=get_db(start_date=start_date,end_date=end_date,cols=["trade_status"])
    df= df[df["trade_status"]==1]
    stk_ls = df["sid"].drop_duplicates()

    results = Parallel(n_jobs=16, verbose=5, backend="loky", batch_size='auto') \
        (delayed(get_period_stk_data)(sid, start_date,end_date) for sid in stk_ls)
    data = pd.concat(results)

    return data.sort_values(["sid", "datetime"])

def append_yesterday_data(df):
    df["yesterday_close"] = df["close"].shift(240)
    return df.iloc[240:]

def data_store(df):
    date=df["DataDate"].iloc[0]
    df = df.drop(["DataDate"],axis=1)
    df.to_csv(r"/home/ywang/proj_cross_grid/result/cross_count_b240/crosscount_{}.csv".format(date),index=False)
    return 0

def get_period_cross_data(start_date, end_date):
    data=get_period_data(get_previous_trade_date(start_date),end_date)
    
    data = data.groupby("sid",as_index = False).apply(append_yesterday_data)

    groups = data.groupby(["sid","DataDate"])
    cross_count_df = apply_parallel(groups, get_count_parallel)
    cross_count_df = cross_count_df.reset_index().drop(["sid"],axis=1).rename(columns={"level_0":"sid","level_1":"DataDate"})
    cross_count_df.groupby("DataDate").apply(data_store)
    return cross_count_df

In [ ]:
data=get_period_data(get_previous_trade_date("20200507"),"20200508")

data = data.groupby("sid",as_index = False).apply(append_yesterday_data)

groups = data.groupby(["sid","DataDate"])
cross_count_df = apply_parallel(groups, get_count_parallel)
cross_count_df = cross_count_df.reset_index().drop(["sid"],axis=1).rename(columns={"level_0":"sid","level_1":"DataDate"})
cross_count_df.groupby("DataDate").apply(data_store)

In [ ]:
get_period_cross_data("20200507","20200508")

In [ ]:
data=get_period_data("20191220","20200131")

data = data.groupby("sid",as_index = False).apply(append_yesterday_data)

groups = data.groupby(["sid","DataDate"])
cross_count_df = apply_parallel(groups, get_count_parallel)
cross_count_df = cross_count_df.reset_index().drop(["sid"],axis=1).rename(columns={"level_0":"sid","level_1":"DataDate"})
cross_count_df.groupby("DataDate").apply(data_store)

In [ ]:
date_ls = get_trade_dates('20180101','20191231')
for i in range(0,len(date_ls),60):
    try:
        get_period_cross_data(date_ls[i],date_ls[i+59])
    except IndexError:
        get_period_cross_data(date_ls[i],'20191231')

In [50]:
#written by wangyonghao

#the api: get_crosscount_summary(date,period_ls)

#input: date: current date ( "20190102" for example)
#       period_ls:the list for the window period ([5,10,15] for example)


#output to file: the excel file of the cross_grid_count_summary for each window_period and for each stock



import numpy as np
import pandas as pd
import tushare as ts
import click

def factor_data_calcu(td_date, stock_ls, window_period, factor_df, crosscount_df):
    date_idx = date_ls.index(td_date)
    date_range = date_ls[date_idx - window_period + 1:date_idx + 1]

    count = np.sum(np.array(crosscount_df.loc[date_range, stock_ls]), axis=0)

    factor_df.loc[stock_ls, "{}d_count".format(window_period)] = count

def get_crosscount_summary(date,period_ls):

    date_range = date_ls[max(1, date_ls.index(date) - 29):date_ls.index(date) + 1]
    crosscount_df = pd.DataFrame()
    for datadate in date_range:
        temp_df = pd.read_csv(r"/home/ywang/proj_cross_grid/result/cross_count_b240/crosscount_{}.csv".format(datadate), index_col="sid")
        temp_df = temp_df.reindex(pd.MultiIndex.from_product([[datadate], temp_df.index]), level=1)
        crosscount_df = pd.concat([crosscount_df, temp_df])
    crosscount_df = crosscount_df.unstack()["cross_count"]
    crosscount_df = crosscount_df.dropna(axis=1, how="any")

    factor_df = pd.DataFrame(index=crosscount_df.columns,
                             columns=[str(i)+"d_count" for i in period_ls])

    for window_period in period_ls:
        factor_data_calcu(date, crosscount_df.columns, window_period, factor_df, crosscount_df)
    factor_df["mean_5-mean_30"]=factor_df["5d_count"]/5-factor_df["30d_count"]/30
    factor_df["mean_10-mean_30"]=factor_df["10d_count"]/10-factor_df["30d_count"]/30

    factor_df.to_excel(r"/home/ywang/proj_cross_grid/result/cross_count_b240/crosscount_summary_{}.xlsx".format(date),
                       index_label="sid")
    
    pre_date = date
    stock_pool=pd.read_csv(r"/share/xfzhang/to_colleague/to_yzhao/task2/{}/{}/task2_{}.csv".\
                                        format(pre_date[0:4],pre_date[4:6],pre_date[0:8]))["sid"]
    factor_df_baima=factor_df.loc[stock_pool]
    factor_df_baima.to_excel(r"/home/ywang/proj_cross_grid/result/cross_count_b240_baima/crosscount_summary_{}.xlsx"\
                             .format(date),index_label="sid")
    

    print(date,"finished")


In [51]:
date_ls=get_trade_dates(20200325,20200521)
date_ls=[str(i) for i in date_ls]
for date in date_ls[29:]:
    get_crosscount_summary(date,[5,10,15,20,30])

/home/ywang/env/dev/lib/python3.6/site-packages/pandas/core/indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)
/home/ywang/env/dev/lib/python3.6/site-packages/ipykernel_launcher.py:51: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike


20200511 finished
20200512 finished
20200513 finished
20200514 finished
20200515 finished
20200518 finished
20200519 finished
20200520 finished
20200521 finished


In [ ]:
pd.read_excel(r"/home/ywang/proj_cross_grid/result/cross_count_b240/crosscount_summary_{}.xlsx".format('20191212'))

In [ ]:
#统计 10d_ma_crosscount 大于10的股票

date_ls=get_trade_dates(20190301,20191029)

summary_df=pd.DataFrame()
for date in date_ls:
    temp_df=pd.read_excel(r"/share/intern_share/stk_bundle/crosscount_summary_{}.xlsx".format(date),index_col="sid")
    temp_df = temp_df.reindex(pd.MultiIndex.from_product([[date], temp_df.index]), level=1)
    for window in [5,10,15,20,30]:
        temp_df["{}d_ma>10".format(window)]=temp_df["{}d_count".format(window)]>10*window
    
    columns=["{}d_ma>10".format(i) for i in [5,10,15,20,30]]
    temp_df["any_ma>10"]=temp_df[columns].apply(lambda x:any(x),axis=1)
    summary_df=pd.concat([summary_df,temp_df])
    
summary_df=summary_df.reset_index().rename(columns={"level_0":"date";,"level_1":"sid"})

sid_date_df=summary_df[["sid","date"]][summary_df["any_ma>10"] == True]
sid_date_df.reset_index(drop=True,inplace=True)
sid_date_df.to_csv(r"data/crosscount_ma_>10_stocklist.csv",index=False)

sid_date_df.to_csv(r"/share/intern_share/assign_1030/crosscount_ma_>10_stocklist.csv",index=False)

In [ ]:
%run cross_count_summary_v240.py "20200115" 5 10 15 20 30